# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

# Setup env

In [7]:
load_dotenv()

OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL")
OLLAMA_MODEL = os.getenv("LLAMA3_3B")

if (OLLAMA_BASE_URL and OLLAMA_MODEL) is None:
    print("One or more environment variales are missing, check - OLLAMA_BASE_URL & LLAMA3_3B.")
else:
    print("Configuration looks good.")
    print(f"Base URL: {OLLAMA_BASE_URL} \nOllama Model: {OLLAMA_MODEL}")

Configuration looks good.
Base URL: http://localhost:11434/v1 
Ollama Model: llama3.2:3b


In [3]:
# PROMPT
code_assistand_system_prompt = """
    You are a helpful coding assistant, expertise in java, python and other scripting and programming languages.
    You help users by providing insight about their code, mention the programming laguage name and then analyse and explain it. 
    Also highlight bugs and suggest improvements.
"""

question = """
    Please explain what this code does and why:
    yield from {book.get("author") for book in books if book.get("author")}
"""

payload = [
    {'role': 'system', 'content': code_assistand_system_prompt},
    {'role': 'user', 'content': question},
]

In [4]:
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="dummy-key")

In [8]:
stream_resp = ollama.chat.completions.create(model=OLLAMA_MODEL, messages=payload, stream=True)

display_handle = display(Markdown(""), display_id=True)
resp = ""
for chunk in stream_resp:
    resp += chunk.choices[0].delta.content or ''
    update_display(Markdown(resp), display_id=display_handle.display_id)


**Code Analysis**

The given code snippet is written in Python. It's using a feature called list comprehension within a `yield` statement, which combines elements of generators and list comprehensions.

Here's what it does in detail:

1. `{book.get("author") for book in books if book.get("author")}`: This part is a list comprehension that iterates over the `books` iterable (potentially a list or other iterable) and extracts values from each dictionary within it.

2. `.get("author")`: It uses the dictionary's `get` method to safely retrieve the value associated with the key `"author"` from each dictionary. If the key does not exist in the dictionary, it returns `None`.

3. `yield from ...`: This is where things get interesting. When combined with list comprehension, `yield from` turns a list comprehension into a generator. A generator is a special type of iterable that can be used to generate values on-the-fly without having to store them all in memory at once.

4. The overall expression: `yield from {book.get("author") for book in books if book.get("author")}` essentially yields each extracted author's name one by one, regardless of whether they exist or not.

So, what does this code do?

**Example Use Case**

Suppose you have a list of dictionaries representing books. Each dictionary might look something like this:

```python
[
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "publisher": "Publisher X"},
    {"title": "Book 3", "author": "Author B"}
]
```

If you run the given code on this list of dictionaries, it would yield `'Author A'` followed by `None` (because there is no `author` key in the second dictionary), and finally `'Author B'`.

**Bugs and Suggestions for Improvement**

This code could raise a TypeError if any of the values from `books` are not iterable. You might want to add some error checking or use the `next()` function with an iterator directly to prevent this.

For instance:

```python
author_iterable = (book["author"] for book in books)
for author in authors:
    author_value = next(author_iterable, None)
```

Alternatively, to deal with any non-iterable values while still preserving the results of existing ones and providing None when those should be excluded due to dictionary access issues:

```python
def filtered_iterators(*args):
    for value in args[0]:
        if isinstance(value, dict) and ("author" in value): 
            yield from (value["author"] for book in args[1] if not (book and "author" in book))
        else:
            yield None

books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "publisher": "Publisher X"},
    {"title": "Book 3", "author": "Author B"}
]

filtered_authors = filtered_iterators(*zip(*[book for book in books if ("author" in book)]))
for author_name in filtered_authors:
    print(author_name)
```

This code checks whether each dictionary has the `author` key and yields dictionaries with their corresponding authors; otherwise, it continues until all values have been iterated over or encountered a non-dictionary value.